# Command visualization app

Note: this notebook is also available in [Google Colaboratoy](https://colab.research.google.com/github/paranal-sw/collaborations-UFRO-2024-2s/blob/main/1-command-visualization/task_description.ipynb)

The VLT Software has a mechanism to send command between processes. This is normally started in the instrument (PIONIER, MATISSE, GRAVITY) and in some cases like template ACQ or template OBS, the command is sent to subsystems (ISS, DL, ATx, ARAL, etc)

It would be useful to have a graphical tool to understands who initiated the messages, the trip it makes through the systems, and included their parameters whenever is possible.

## Objective
1. Create a streamlit app to display command messages through processes.
1. Fill the gaps when commands send and receive are not clearly written.
1. Report the process to create the app and the assumtpioms made to fill the gaps.

Make a UML sequence diagram with the time(start and end) between process(if it fails or not), the instrument, the command(name) and the susbsystem(name), 

## Example app

A sample streamlit app is provided to use as barebones. Please create subpages with different versions.

To start the app:

```bash
cd collaborations-UFRO-2024-2s
source venv/bin/activate

cd 1-command-visualization
streamlit run Home_example1.py
```

In [1]:
import os 
import pandas as pd
from urllib.request import urlretrieve
REPO_URL='https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    
    PATH='data/' # Convenient name to be Colab compatible
else:
    PATH='D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data' # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):

    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst=pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs=pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele=pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[ df_all['trace_id']==trace_id ]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta=pd.read_parquet(f'{PATH}/{fname}')

    return df_meta

## Example of command events

In [2]:
df_meta = load_meta("MATISSE", "1w")
df_meta

,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,SECONDS,TEL
0,2019-04-02 01:29:45.681,2019-04-02 01:38:08.541,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,502.0,AT
1,2019-04-02 01:38:08.559,2019-04-02 01:44:02.506,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,353.0,AT
2,2019-04-02 01:45:44.711,2019-04-02 01:58:00.921,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,736.0,AT
3,2019-04-02 01:58:19.863,2019-04-02 02:14:00.315,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,940.0,AT
4,2019-04-02 06:36:28.631,2019-04-02 07:00:16.566,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,1427.0,AT
...,...,...,...,...,...,...,...,...,...,...
215,2019-04-07 21:20:07.817,2019-04-07 21:22:12.808,False,MATISSE,bob_ins,MATISSE_gen_tec_opd,False,False,124.0,AT
216,2019-04-07 21:22:25.249,2019-04-07 21:34:42.495,False,MATISSE,bob_ins,MATISSE_gen_tec_check_fringes,False,False,737.0,AT
217,2019-04-07 21:34:42.495,2019-04-07 21:39:06.173,False,MATISSE,bob_ins,MATISSE_gen_cal_bias,False,False,263.0,AT
218,2019-04-07 21:40:15.747,2019-04-07 21:42:02.235,False,MATISSE,bob_ins,MATISSE_gen_cal_bias,False,False,106.0,AT


In [3]:
df_meta['Aborted'].value_counts()

Aborted
False    201
True      19
Name: count, dtype: int64

In [4]:
df_meta['ERROR'].value_counts()

ERROR
False    214
True       6
Name: count, dtype: int64

In [5]:
df_meta['TIMEOUT'].value_counts()



TIMEOUT
False    220
Name: count, dtype: int64

In [6]:
df_meta['TEL'].value_counts()

TEL
AT    220
Name: count, dtype: int64

In [7]:
df_meta['TPL_ID'].value_counts()

TPL_ID
MATISSE_img_acq                  43
MATISSE_gen_tec_opd              42
MATISSE_hyb_obs                  26
MATISSE_gen_cal_bias             21
MATISSE_gen_cal_imbasic          18
MATISSE_gen_cal_det_h2           17
MATISSE_gen_cal_shift            16
MATISSE_gen_cal_transfunc         6
MATISSE_gen_tec_standby           6
MATISSE_gen_tec_ali               6
MATISSE_gen_cal_kappa             6
MATISSE_gen_tec_check_fringes     5
CALOB_gen_tec_log                 3
MATISSE_gen_cal_det_aq            2
MATISSE_gen_cal_imrem             2
MATISSE_gen_cal_imext             1
Name: count, dtype: int64

In [8]:
df_trace = load_trace("MATISSE", "1w", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]

### Search *command*


In [9]:
with pd.option_context('display.max_colwidth', None):
    display(df_trace[ df_trace['logtext'].str.contains("command") ][:50])

,@timestamp,system,procname,logtext,trace_id
131,2019-04-02 01:29:47.435,MATISSE,mtoControl,Executing SETUP command ...,0
135,2019-04-02 01:29:47.440,MATISSE,mtoControl,SETUP command done.,0
138,2019-04-02 01:29:47.696,MATISSE,mtoControl,Executing SETUP command ...,0
164,2019-04-02 01:30:09.566,MATISSE,mtoControl,SETUP command done.,0
167,2019-04-02 01:30:09.567,MATISSE,mtoControl,Executing SETUP command ...,0
172,2019-04-02 01:30:11.165,MATISSE,mtoControl,SETUP command done.,0
175,2019-04-02 01:30:11.166,MATISSE,mtoControl,Executing SETUP command ...,0
180,2019-04-02 01:30:13.374,MATISSE,mtoControl,SETUP command done.,0
182,2019-04-02 01:30:13.397,MATISSE,bob_ins,"Send command CHKMOON 064508.910,-164258.000,0.0 to process issifControl env. wvgvlti",0
183,2019-04-02 01:30:13.418,ISS,issifControl,"CDP : Received command: CHKMOON, Buffer: 064508.910,-164258.000,0.0",0


In [10]:
with pd.option_context('display.max_colwidth', None):
    display(df_trace[ df_trace['logtext'].str.contains("CHKMOON") ][:50])



,@timestamp,system,procname,logtext,trace_id
182,2019-04-02 01:30:13.397,MATISSE,bob_ins,"Send command CHKMOON 064508.910,-164258.000,0.0 to process issifControl env. wvgvlti",0
183,2019-04-02 01:30:13.418,ISS,issifControl,"CDP : Received command: CHKMOON, Buffer: 064508.910,-164258.000,0.0",0
184,2019-04-02 01:30:13.419,MATISSE,bob_ins,Waiting (timeout 40000) for pending replies to command CHKMOON id cmd0,0
185,2019-04-02 01:30:13.482,ISS,issifControl,"CDP : Succesfully completed command: CHKMOON, Buffer: OK",0
194,2019-04-02 01:30:14.440,MATISSE,bob_ins,Last reply to command CHKMOON id cmd0 received,0


In [11]:
df_tratamiento=df_trace[ df_trace['logtext'].str.contains("CHKMOON") ][:]
#print(df_tratamiento['system'])
#print(df_tratamiento['procname'])
#print(df_tratamiento[0:1]['system'])
df_tratamiento.head()

,@timestamp,system,procname,logtext,trace_id
182,2019-04-02 01:30:13.397,MATISSE,bob_ins,"Send command CHKMOON 064508.910,-164258.000,0....",0
183,2019-04-02 01:30:13.418,ISS,issifControl,"CDP : Received command: CHKMOON, Buffer: 064...",0
184,2019-04-02 01:30:13.419,MATISSE,bob_ins,Waiting (timeout 40000) for pending replies to...,0
185,2019-04-02 01:30:13.482,ISS,issifControl,CDP : Succesfully completed command: CHKMOON...,0
194,2019-04-02 01:30:14.440,MATISSE,bob_ins,Last reply to command CHKMOON id cmd0 received,0


In [12]:
df_tratamiento[df_tratamiento['logtext'].str.contains("Send")]['system']

182    MATISSE
Name: system, dtype: object

In [13]:
df_tratamiento[df_tratamiento['logtext'].str.contains("Received command")]['system']

183    ISS
Name: system, dtype: object

In [14]:
df_tratamiento[df_tratamiento['logtext'].str.contains("Waiting")]['system']

184    MATISSE
Name: system, dtype: object

In [15]:
df_tratamiento[df_tratamiento['logtext'].str.contains("Succesfully completed")]['system']


185    ISS
Name: system, dtype: object

In [16]:
df_tratamiento[df_tratamiento['logtext'].str.contains("Last reply")]['system']

194    MATISSE
Name: system, dtype: object

In [17]:
with pd.option_context('display.max_colwidth', None):
    display(df_trace[ df_trace['logtext'].str.contains("CLRSTP") ][:])

,@timestamp,system,procname,logtext,trace_id
241,2019-04-02 01:30:17.668,MATISSE,bob_ins,Send command CLRSTP to process issifControl env. wvgvlti,0
242,2019-04-02 01:30:17.670,MATISSE,bob_ins,Waiting (timeout 15000) for pending replies to command CLRSTP id cmd0,0
243,2019-04-02 01:30:17.670,ISS,issifControl,"CDP : Received command: CLRSTP, Buffer:",0
244,2019-04-02 01:30:17.672,AT1,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
245,2019-04-02 01:30:17.672,AT1,atifControl,"TIF : Received command: CLRSTP, Buffer:",0
246,2019-04-02 01:30:17.673,AT2,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
247,2019-04-02 01:30:17.673,AT2,atifControl,"TIF : Received command: CLRSTP, Buffer:",0
248,2019-04-02 01:30:17.675,AT3,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
249,2019-04-02 01:30:17.675,AT3,atifControl,"TIF : Received command: CLRSTP, Buffer:",0
250,2019-04-02 01:30:17.676,AT4,atifControl,"TIF : Received command: CLRSTP, Buffer:",0


### List of commands

This method must be double checked

In [18]:
cmd = df_trace[ df_trace['logtext'].str.contains("command")]['logtext']

# Please replace this with an elegant regexp!
cmd = cmd.apply(lambda x: x.replace(":", " "))
cmd = cmd.apply(lambda x: x.replace(",", " "))
cmd = cmd.apply(lambda x: x.replace("'", " "))
cmd = cmd.apply(lambda x: x.replace("  ", " "))

cmdlist = []
for x in cmd:
    splitted = x.split()
    # print(splitted)
    if "command" in splitted:
        idx = splitted.index("command")
        command = splitted[idx+1]
        if command not in ["...", "done.", "to"]:
            cmdlist.append(command)
set(cmdlist)

{'CHKMOON',
 'CLRSTP',
 'GETCONF',
 'GETDEL',
 'GETDELD',
 'GETIAS',
 'PRABS',
 'PRCHK',
 'PRESET',
 'PRSCOOR',
 'PRSNAME',
 'PRTOGS',
 'SETBEAM',
 'SETPOF',
 'SETREF',
 'SETTADC',
 'SETUP',
 'STOPAG',
 'STOPCAG',
 'STOPCHP',
 'STOPLAG',
 'STRTLAG'}

### Search *SETUP*


In [19]:
with pd.option_context('display.max_colwidth', None):
    display(df_trace[ df_trace['logtext'].str.contains("SETUP") ][:])

,@timestamp,system,procname,logtext,trace_id
130,2019-04-02 01:29:47.434,MATISSE,bob_ins,SETUP -noExposure -function DET1.START.VAL 1 DET2.START.VAL 1 (blue),0
131,2019-04-02 01:29:47.435,MATISSE,mtoControl,Executing SETUP command ...,0
132,2019-04-02 01:29:47.436,MATISSE,mtoControl,Forward(b) SETUP to NGCIR1,0
133,2019-04-02 01:29:47.436,MATISSE,mtoControl,Forward(b) SETUP to NGCIR2,0
135,2019-04-02 01:29:47.440,MATISSE,mtoControl,SETUP command done.,0
...,...,...,...,...,...
2359,2019-04-02 01:38:00.099,MATISSE,mtoControl,Executing SETUP command ...,0
2373,2019-04-02 01:38:00.488,MATISSE,mtoControl,SETUP command done.,0
2374,2019-04-02 01:38:00.489,MATISSE,bob_ins,SETUP -noExposure -function INS.SOS1.NAME OUT INS.SOS2.NAME OUT INS.BCD1.NAME OUT INS.BCD2.NAME OUT INS.OML1.OPD 0 INS.OML2.OPD 0 INS.OML3.OPD 0 INS.OML4.OPD 0,0
2377,2019-04-02 01:38:00.489,MATISSE,mtoControl,Executing SETUP command ...,0


In [20]:
with pd.option_context('display.max_colwidth', None):
    display(df_trace[ df_trace['logtext'].str.contains("SETUP command") ][:50])

,@timestamp,system,procname,logtext,trace_id
131,2019-04-02 01:29:47.435,MATISSE,mtoControl,Executing SETUP command ...,0
135,2019-04-02 01:29:47.440,MATISSE,mtoControl,SETUP command done.,0
138,2019-04-02 01:29:47.696,MATISSE,mtoControl,Executing SETUP command ...,0
164,2019-04-02 01:30:09.566,MATISSE,mtoControl,SETUP command done.,0
167,2019-04-02 01:30:09.567,MATISSE,mtoControl,Executing SETUP command ...,0
172,2019-04-02 01:30:11.165,MATISSE,mtoControl,SETUP command done.,0
175,2019-04-02 01:30:11.166,MATISSE,mtoControl,Executing SETUP command ...,0
180,2019-04-02 01:30:13.374,MATISSE,mtoControl,SETUP command done.,0
233,2019-04-02 01:30:17.534,MATISSE,mtoControl,Executing SETUP command ...,0
237,2019-04-02 01:30:17.659,MATISSE,mtoControl,SETUP command done.,0


### Search *PRESET*


In [21]:
with pd.option_context('display.max_colwidth', None):
    display(df_trace[ df_trace['logtext'].str.contains("PRESET") ][:59])

,@timestamp,system,procname,logtext,trace_id
79,2019-04-02 01:29:45.947,MATISSE,bob_ins,SEQ PRESET ST = 'T',0
244,2019-04-02 01:30:17.672,AT1,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
246,2019-04-02 01:30:17.673,AT2,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
248,2019-04-02 01:30:17.675,AT3,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
251,2019-04-02 01:30:17.676,AT4,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
252,2019-04-02 01:30:17.677,DL,dlprsControl,"PRESET: Received command: CLRSTP, Buffer:",0
255,2019-04-02 01:30:17.681,DL,dlprsControl,PRESET: Command CLRSTP (NoMove) succesfully completed,0
258,2019-04-02 01:30:17.736,AT3,prsControl,PRESET: Command STOPTRK (NoMove) succesfully completed,0
260,2019-04-02 01:30:17.741,AT2,prsControl,PRESET: Command STOPTRK (NoMove) succesfully completed,0
263,2019-04-02 01:30:17.747,AT1,prsControl,PRESET: Command STOPTRK (NoMove) succesfully completed,0


In [22]:
# import pandas as pd
# # the location of the files data
# file_path = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data/PIONIER-1w-meta.parket'
# # read the dataframe from the parquet file
# df = pd.read_parquet(file_path)
# df



In [23]:
# df_meta = load_meta("PIONIER", "1w")
# df_meta

In [24]:
df_trace1=df_trace[df_trace['logtext'].str.contains("")][0:1000]
df_trace1.to_excel('df_trace1.xlsx')

In [25]:
import os 
import pandas as pd
from urllib.request import urlretrieve
REPO_URL='https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH='data/' # Convenient name to be Colab compatible
else:
    PATH='D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data' # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_all_traces(RANGE):
    instruments = ["MATISSE", "PIONIER", "GRAVITY"]  # Añade todos los instrumentos aquí
    all_dfs = []
    for instrument in instruments:
        df_all = load_dataset(instrument, RANGE)
        all_dfs.append(df_all)
    
    df_all_instruments = pd.concat(all_dfs)
    df_all_instruments.sort_values('@timestamp', inplace=True)
    df_all_instruments.reset_index(drop=True, inplace=True)
    
    return df_all_instruments

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[ df_all['trace_id']==trace_id ]
    return df_all


def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')

    return df_meta

In [26]:
df_trace = load_all_traces( "1w")
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]

with pd.option_context('display.max_colwidth', None):
    display(df_trace[df_trace['@timestamp'].between('2019-04-02 00:00:00', '2019-04-07 23:59:59')])
#df_trace.to_excel('trace.xlsx')

,@timestamp,system,procname,logtext,trace_id
43,2019-04-02 01:29:45.681,MATISSE,bob_ins,Started at 2019-04-02T01:29:45 (underlined),0
44,2019-04-02 01:29:45.681,MATISSE,bob_ins,MATISSE_img_acq -- Celestial target acquisition (yellow),0
45,2019-04-02 01:29:45.945,MATISSE,bob_ins,COU AG PMD = '0.0',0
46,2019-04-02 01:29:45.945,MATISSE,bob_ins,COU AG EQUINOX = '2000.0',0
47,2019-04-02 01:29:45.945,MATISSE,bob_ins,DET1 APOY VAL = '512',0
...,...,...,...,...,...
1991389,2019-04-07 23:57:56.941,GRAVITY,bob_44144,ERROR performing command SETUP (INFO: Preset telescopes to FREE position ... (red),151
1991390,2019-04-07 23:57:56.941,GRAVITY,bob_44144,Finished in 14 seconds at 2019-04-07T23:57:56 (underlined),151
1991391,2019-04-07 23:57:56.941,GRAVITY,bob_44144,PRESET). (red),151
1991392,2019-04-07 23:57:56.941,GRAVITY,bob_44144,INFO: Stop Coude auto guiding ... (red),151


In [27]:
df_trace = load_all_traces("1d")
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]

# Filtrar por trace_id igual a 0
df_trace_filtered = df_trace[df_trace['trace_id'] == 1]

# Filtrar por el rango de fechas
#df_trace_filtered = df_trace_filtered[df_trace_filtered['@timestamp'].between('2019-04-02 00:00:00', '2019-04-07 23:59:59')]

with pd.option_context('display.max_colwidth', None):
    display(df_trace_filtered)

,@timestamp,system,procname,logtext,trace_id
2238,2019-04-10 00:20:35.848,MATISSE,bob_ins,MATISSE_hyb_obs -- Celestial target observation (yellow),1
2239,2019-04-10 00:20:35.848,MATISSE,bob_ins,Started at 2019-04-10T00:20:35 (underlined),1
2240,2019-04-10 00:20:36.000,AT3,logManager,Motion execution.,1
2241,2019-04-10 00:20:36.111,MATISSE,bob_ins,INS FIL NAME = 'AUTO',1
2242,2019-04-10 00:20:36.111,MATISSE,bob_ins,DET1 WLMAXCUT VAL = '4.8',1
...,...,...,...,...,...
305577,2019-04-10 11:03:06.817,GRAVITY,bob_ins,-1 (SpringGreen4),1
305578,2019-04-10 11:03:06.817,GRAVITY,bob_ins,"Template P2VM-WAVESC - Woll: IN, Grism: MED, DET1: 0.7, DET2: 0.3 finished. (brown)",1
305579,2019-04-10 11:03:06.817,GRAVITY,gvoControl,SETUP command done.,1
305580,2019-04-10 11:03:06.819,GRAVITY,bob_ins,Finished in 2342 seconds at 2019-04-10T11:03:06 (underlined),1


In [28]:

# Contar el número de trace_id igual a 0
count_trace_id_0 = (df_trace['trace_id'] == 0).sum()

print(f"Number of trace_id values equal to 0: {count_trace_id_0}")

Number of trace_id values equal to 0: 8261


# Necesitamos:

1. como identificar como empieza y termina un proceso
2. como identificar cuando empieza un comando y cuando termina